<a href="https://colab.research.google.com/github/akaicomet/NJOITOpenDataCenter-Pension-Record/blob/master/NJOITOpenDataCenter_Pension_Record_ModelBuilding_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install sodapy
!pip install matplotlib
!pip install hyperopt
!pip install bayesian-optimization
!pip install tbvaccine 
!pip install MulticoreTSNE
!pip install -U -q PyDrive

    100% |████████████████████████████████| 61kB 5.4MB/s 
    100% |████████████████████████████████| 829kB 13.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


  Stored in directory: /root/.cache/pip/wheels/4b/60/a2/6eec4adc73341c1a2a653022b99ff8e1cb090684dba83be683
Successfully built bayesian-optimization
  Stored in directory: /root/.cache/pip/wheels/27/59/53/3b52ee63add3692254c30d687fa4dff4d128d0557861fb028e
Successfully built MulticoreTSNE
    100% |████████████████████████████████| 993kB 21.0MB/s 


#1. Copy python files from  GItHub repository to local

In [2]:
!git clone https://github.com/akaicomet/NJOITOpenDataCenter.git
!git clone https://github.com/akaicomet/akaicomet_Utility.git
!git clone https://github.com/akaicomet/akaicomet_Chart.git
!git clone https://github.com/akaicomet/akaicomet_ML.git

Cloning into 'NJOITOpenDataCenter'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 108 (delta 62), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (108/108), 11.01 MiB | 7.91 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Cloning into 'akaicomet_Utility'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Cloning into 'akaicomet_Chart'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 73 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.
Cloning into 'akaicomet_ML'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
r

#2. Run .py files in local

In [0]:
%run -i /content/akaicomet_Utility/ErrorHandling.py

%run -i /content/NJOITOpenDataCenter/YourMoneyActivePensionMembers.py 
%run -i /content/NJOITOpenDataCenter/YourMoneyRetiredPensionMembers.py
%run -i /content/NJOITOpenDataCenter/YourMoneyPensionMembersMerge.py

%run -i /content/akaicomet_Chart/X-Ray_Scan.py
%run -i /content/akaicomet_Chart/akaicomet_Chart.py

%run -i /content/akaicomet_ML/ML_Init.py
%run -i /content/akaicomet_ML/ML_Auto.py
%run -i /content/akaicomet_ML/AutoFeatureEngineering.py


#3. Import libraries

In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import random
from sodapy import Socrata
#from pandas.plotting import scatter_matrix
#from datetime import datetime
import seaborn as sb
from tqdm import tqdm
import warnings
from textwrap import wrap

In [0]:
from sklearn.externals import joblib
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#4. YourMoney and the NJOIT Open Data Center 
##4.1 YourMoney Active Pension Members API

**Pension member data load from NJOIT** 

In [6]:
client = Socrata("data.nj.gov", None)
results = client.get("45bd-gwii", limit=100000000)#100000000

**Data Cleansing and Preparation for YourMoney Active Pension Members data**

In [0]:
ActivePension_df = ActivePensionDataWrangler(results)

##4.2 YourMoney Retired Pension Members API
**Pension payment member data load from NJOIT**

In [0]:
results = client.get("8up4-62p6",   limit=100000000)

**Data Cleansing and Preparation for YourMoney Retired Pension data**

In [0]:
RetiredPension_df_grped = RetiredPensionMembersDataWrangler(results)

**Merge YourMoney Active Pension Members data and YourMoney Retired Pension data**

In [0]:
ActivePension_df = PensionMembersMerge(ActivePension_df,RetiredPension_df_grped)
del results

## 5 Recheck  Correlation Variable Validation:

In [11]:
#x_cols2 = ['_20_year_status',	'all_employers_salary_amt',		'employer_freq_pensioned', 'total_months_qty',	'location_freq_pensioned',		'current_employer_salary_rollingamt', 	'service_months_qty',	'pension_freq_pensioned', 'pension_fund_name_enc', 'pension_group_name_enc' ]
x_cols2 = ['_20_year_status','all_employers_salary_amt','employer_freq_pensioned','total_months_qty','location_freq_pensioned','service_months_qty','pension_freq_pensioned','pension_fund_name_enc','pension_group_name_enc' ]
ActivePension_df[x_cols2].corr()

,_20_year_status,all_employers_salary_amt,employer_freq_pensioned,total_months_qty,location_freq_pensioned,service_months_qty,pension_freq_pensioned,pension_fund_name_enc,pension_group_name_enc
_20_year_status,1.000000,0.295118,-0.074679,0.844411,0.082618,-0.002429,0.091064,-0.238068,-0.113550
all_employers_salary_amt,0.295118,1.000000,-0.066198,0.395345,-0.030279,-0.027708,-0.439474,0.207998,0.097105
employer_freq_pensioned,-0.074679,-0.066198,1.000000,-0.065524,0.187599,0.026186,0.040389,0.099662,-0.033238
total_months_qty,0.844411,0.395345,-0.065524,1.000000,0.067168,0.000102,0.047303,-0.187378,-0.121364
location_freq_pensioned,0.082618,-0.030279,0.187599,0.067168,1.000000,0.056022,0.308507,0.047458,-0.063150
service_months_qty,-0.002429,-0.027708,0.026186,0.000102,0.056022,1.000000,0.079330,0.052583,-0.034342
pension_freq_pensioned,0.091064,-0.439474,0.040389,0.047303,0.308507,0.079330,1.000000,-0.306339,-0.308736
pension_fund_name_enc,-0.238068,0.207998,0.099662,-0.187378,0.047458,0.052583,-0.306339,1.000000,0.185796
pension_group_name_enc,-0.113550,0.097105,-0.033238,-0.121364,-0.063150,-0.034342,-0.308736,0.185796,1.000000


# 6 Load clfs Data from Google Drive
##  6.1 Join Table 

In [12]:
#Need to change 'id' everytime

downloaded = drive.CreateFile({'id':'1IU4Dxszpgxz9vJs8hH9Nz5v5ClPKiFPA'})
downloaded.GetContentFile('clfs_score612.pkl')

downloaded = drive.CreateFile({'id':'1qlumiKj10YDBFX4s2jxCTTSx36hUOplm'})
downloaded.GetContentFile('clfs_score623.pkl')

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [0]:
#downloaded = drive.CreateFile({'id':'1vA1Z9Y9OkTNp83XR3WLuAyA-HemgTErA'})
#downloaded.GetContentFile('clfs612.pkl')
#clfs = joblib.load('clfs612.pkl')

In [0]:
clfs_score612 = joblib.load('clfs_score612.pkl')
clfs_score623 = joblib.load('clfs_score623.pkl')

In [0]:
df_clfs_score = pd.DataFrame.from_dict(clfs_score612,orient='index')

In [0]:
df_clfs_score2 = pd.DataFrame.from_dict(clfs_score623,orient='index')

In [0]:
df_clfs_score = pd.concat([df_clfs_score, df_clfs_score2], axis=1, join='inner')

## 6.2  Additional Field and Manupulation

In [0]:
df_clfs_score['classifier'] = [i.split('|')[0] for i  in df_clfs_score.index.tolist()]

In [0]:
df_clfs_score['std'] = df_clfs_score.std(axis=1)

In [0]:
df_clfs_score.columns = ['Training', 'Test1', 'Test2', 'Test3', 'classifier','std']

In [23]:
df_clfs_score.sort_values(['classifier','std','Training'], ascending=[True, True, True]).groupby('classifier')['std'].nsmallest(10)

,Training,Test1,Test2,Test3,classifier,std
AdaBoostClassifier|MDS MinMaxScaler|Normalizer|,0.929429,0.930000,0.931333,0.921,AdaBoostClassifier,0.004695
AdaBoostClassifier|Raw MinMaxScaler|,0.940857,0.944000,0.942667,0.931,AdaBoostClassifier,0.005896
AdaBoostClassifier|KPCA MinMaxScaler|Normalizer|,0.942286,0.940000,0.942667,0.930,AdaBoostClassifier,0.005943
AdaBoostClassifier|KPCA Normalizer|QuantileTransformer|,0.942286,0.940667,0.942667,0.930,AdaBoostClassifier,0.005999
AdaBoostClassifier|Raw StandardScaler|,0.943143,0.938667,0.942667,0.930,AdaBoostClassifier,0.006087
AdaBoostClassifier|Raw QuantileTransformer|,0.941143,0.942667,0.942667,0.930,AdaBoostClassifier,0.006122
AdaBoostClassifier|KPCA PowerTransformer|QuantileTransformer|,0.941143,0.942667,0.942667,0.930,AdaBoostClassifier,0.006122
AdaBoostClassifier|PCA MinMaxScaler|Normalizer|,0.943429,0.937333,0.942667,0.930,AdaBoostClassifier,0.006196
AdaBoostClassifier|PCA StandardScaler|QuantileTransformer|,0.940857,0.943333,0.942667,0.930,AdaBoostClassifier,0.006231
AdaBoostClassifier|Raw MinMaxScaler|PowerTransformer|,0.940857,0.944000,0.942667,0.930,AdaBoostClassifier,0.006385


## 6.3 Refine Redimensioning to Validate Model 

In [24]:
manager = Manager()
clfs3 = manager.dict()
clfs_score3 = manager.dict()

flatten = lambda l: [item for sublist in l for item in sublist]

y_train = ActivePension_df.sample(n=5000, random_state=2)['IsPensionPaid'].as_matrix()
X_train = ActivePension_df[x_cols2].sample(n=5000, random_state=2)

X_train = X_train.astype(float)

scaling_def = {'MinMaxScaler': MinMaxScaler(),
               'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),
               'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False),
               'Normalizer_l2': Normalizer(norm='l2'), 'Normalizer_l1': Normalizer(norm='l1'),
               'PowerTransformer_std':PowerTransformer(standardize=True),'PowerTransformer':PowerTransformer(standardize=False)
              
              }


redemensioning_def = {'Raw':None,
                      'PCA': PCA(n_components=0.99, svd_solver='full',whiten=False),'PCAW': PCA(n_components=0.99, svd_solver='full',whiten=True),
                      'ICA_P': FastICA(n_components=len(X_train.columns), algorithm='parallel'),'ICA_D': FastICA(n_components=len(X_train.columns), algorithm='deflation'),
                      'KPCA_rbf': KernelPCA(n_components=len(X_train.columns), kernel='rbf',n_jobs=-1),'KPCA_lin': KernelPCA(n_components=len(X_train.columns), kernel='linear',n_jobs=-1),'KPCA_pol': KernelPCA(n_components=len(X_train.columns), kernel='poly',n_jobs=-1),'KPCA_sigm': KernelPCA(n_components=len(X_train.columns), kernel='sigmoid',n_jobs=-1),'KPCA_cos': KernelPCA(n_components=len(X_train.columns), kernel='cosine',n_jobs=-1),
                      'MDS_met': MDS(n_components=len(X_train.columns), metric=True, n_jobs=-1),'MDS': MDS(n_components=len(X_train.columns), metric=False, n_jobs=-1)
                     }


for clf in df_clfs_score.index.tolist():
  for i in clf.split(' ')[1:]:
    for j in i.split('|'):
      if j != '':
        scaling_def_temp = {s: scaling_def[s]  for s in scaling_def if j in s}
        redemensioning_def_temp = {i: redemensioning_def[i] for i in flatten([i.split(' ') for i in clf.split('|')])[1:] if i in redemensioning_def}
        for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def_temp), scaling(X_train,scaling_def_temp)):
          for x in data:                  
            #print(x[1])
            #print(x[0])
            p = Process(target=MLModelBuilding, args=(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3))
            #MLModelBuilding(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3)
            p.start()
            p.join(180)
            p.terminate()
              

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9894285714285714
Test score: 0.9406666666666667
Test2 score: 0.9706666666666667
RandomForestClassifier|Raw MinMaxScaler|

Train score: 0.9445714285714286
Test score: 0.9366666666666666
Test2 score: 0.944
GradientBoostingClassifier|Raw MinMaxScaler|

best estimate parameters {'C': 0.5291466378476807, 'class_weight': '', 'fit_intercept': False, 'intercept_scaling': 0.009939277570796379, 'loss': 'hinge', 'max_iter': 87, 'tol': 0.9736301921902784}
Train score: 0.9397142857142857
Test score: 0.9406666666666667
Test2 score: 0.9406666666666667
LinearSVC|Raw MinMaxScaler|

best estimate parameters {'C': 0.5809364371899274, 'intercept_scaling': 0.11976046044066235, 'max_iter': 85, 'solver': 'saga', 'tol': 0.6178361288585833}
Train score: 0.9397142857142857
Test score: 0.9406666666666667
Test2 score: 0.9406666666666667
LogisticRegression2|Raw MinMaxScaler|

best estimate parameters {'learning_rate': 2.0323819474931994, 'n_estimators': 203}
Train score: 0.9411428571428572
Test scor

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9897142857142858
Test score: 0.94
Test2 score: 0.9726666666666667
RandomForestClassifier|PCA MinMaxScaler|

Train score: 0.9442857142857143
Test score: 0.9413333333333334
Test2 score: 0.9433333333333334
GradientBoostingClassifier|PCA MinMaxScaler|

best estimate parameters {'C': 0.02270943032400552, 'class_weight': '', 'fit_intercept': False, 'intercept_scaling': 0.6544403038733598, 'loss': 'squared_hinge', 'max_iter': 62, 'tol': 0.42870669957202856}
Train score: 0.9397142857142857
Test score: 0.9406666666666667
Test2 score: 0.9406666666666667
LinearSVC|PCA MinMaxScaler|

best estimate parameters {'C': 0.24158687301357062, 'intercept_scaling': 0.8852944170232928, 'max_iter': 78, 'solver': 'liblinear', 'tol': 0.4680698849845861}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


...

Train score: 0.9402857142857143
Test score: 0.9393333333333334
Test2 score: 0.9406666666666667
LinearSVC|PCA PowerTransformer_std|PowerTransformer|

best estimate parameters {'C': 0.059556336716874325, 'class_weight': '', 'fit_intercept': False, 'intercept_scaling': 0.6448636004379484, 'loss': 'squared_hinge', 'max_iter': 78, 'tol': 0.8152553458792184}
Train score: 0.9402857142857143
Test score: 0.9393333333333334
Test2 score: 0.9406666666666667
LinearSVC|PCA QuantileTransformer_u|

best estimate parameters {'C': 0.532245790340536, 'class_weight': 'balanced', 'fit_intercept': True, 'intercept_scaling': 0.2208464782975919, 'loss': 'squared_hinge', 'max_iter': 4, 'tol': 0.21147974994349225}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train score: 0.9137142857142857
Test score: 0.9173333333333333
Test2 score: 0.918
LinearSVC|PCA QuantileTransformer_n|

best estimate parameters {'C': 0.9986144397784276, 'class_weight': '', 'fit_intercept': True, 'intercept_scaling': 0.6848297481620492, 'loss': 'hinge', 'max_iter': 12, 'tol': 0.8519931859422311}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC|PCA QuantileTransformer_u|QuantileTransformer_n|

best estimate parameters {'learning_rate': 1.0612899289154218, 'n_estimators': 31}
Train score: 0.9397142857142857
Test score: 0.9353333333333333
Test2 score: 0.938
AdaBoostClassifier|PCA PowerTransformer_std|

best estimate parameters {'learning_rate': 2.2199142360800215, 'n_estimators': 256}
AdaBoostClassifier|PCA PowerTransformer|

best estimate parameters {'learning_rate': 1.3423617146430704, 'n_estimators': 3}
Train score: 0.916
Test score: 0.9146666666666666
Test2 score: 0.914
AdaBoostClassifier|PCA PowerTransformer_std|PowerTransformer|

best estimate parameters {'learning_rate': 1.3885771549660122, 'n_estimators': 92}
Train score: 0.9434285714285714
Test score: 0.938
Test2 score: 0.942
AdaBoostClassifier|PCA QuantileTransformer_u|

best estimate parameters {'learning_rate': 2.0602471485740974, 'n_estimators': 267}
Train score: 0.9402857142857143
Test score: 0.9393333333333334
Test2 score: 0.9406666666666667
AdaBoostCla

In [25]:
clfs32 = clfs3.copy()
clfs_score32 = clfs_score3.copy()

joblib.dump(clfs32, 'clfs624.pkl', compress=True)
joblib.dump(clfs_score32, 'clfs_score624.pkl', compress=True)

['clfs_score624.pkl']

In [29]:
folder_id = '1wRbAVDSYm-NJsRMUqkRsreELnGaNM88R'
file = drive.CreateFile({'mimeType': 'content/clfs624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs624.pkl')
file.Upload() 
file = drive.CreateFile({'mimeType': 'content/clfs_score624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs_score624.pkl')
file.Upload() 

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [0]:
#clfs3 = joblib.load('clfs624.pkl')
#clfs_score3 = joblib.load('clfs_score624.pkl')